In [1]:
import numpy as np
import pprint as pp
import math 

## Empezamos  inicializando los vectores Theta

In [2]:
##Definamos los tamaños de cada capa. La de input sera de 784, la hidden layer de 258, la output de 10 (existen 10 posibles clasificaciones).
layer_sizes={'ilayer':28, 'hlayer':7, 'olayer':10}
##Thetas
#Solo tenemos una hidden layer, por lo que tenemos un theta de I a Z(1), y una de Z(1) a O.
#Les generamos valores aleatorios entre 0 y 1.
thetaI_Z = np.random.rand(layer_sizes['ilayer'], layer_sizes['hlayer'])
thetaZ_O = np.random.rand(layer_sizes['hlayer'], layer_sizes['olayer'])
thetas_dict = {'Theta I_Z': thetaI_Z,
              'Theta Z_O': thetaZ_O}


## Funciones

In [3]:
##Definamos las funciones a utilizar.
def sigmoid(Z):
    #print (1/(1+np.exp(-Z)))
    return (1/(1+np.exp(-Z)))
def sigmoid_derivative(Z):
    #print(Z, 1-Z)
    return (Z)*(1-(Z))
def forward_propagate(X, activation_function=sigmoid):
    ##...hay una discusion por hacerse sobre utilizar diccionarios dinamicos para theta de tamaño variable..
    ##..por ahora los haremos constantes
    
    #print(X)
    #print(thetas_dict['Theta I_Z'])
    Z_1 = np.dot(X, thetas_dict['Theta I_Z'])
    Z_2 = activation_function(Z_1) ##Necesitamos una funcion de activacion, por default usaremos sigmoide.
    #print(Z_2)
    Z_3 = np.dot(Z_2, thetas_dict['Theta Z_O'])
    #print(Z_3)
    activation = sigmoid(Z_3)
    results_dict = {'Z1': Z_1,
                   'Z2': Z_2,
                   'Z3': Z_3,
                   'Activation': activation}
    return results_dict

def backprop(X, y, forward_prop_results, activation_derivative=sigmoid_derivative, debug=False, ld=0.0001):
    ##Desenpaquemos valores
    #Fprop, al final solo usamos estos dos, pero sirve tener lo otro para debug.
    fprop_output = forward_prop_results['Activation']
    fprop_Z2 = forward_prop_results['Z2']
    #Thetas
    #theta_1 = thetas_dict['Theta I_Z']
    #theta_2 =  
    ##Errores de salida
    output_error = fprop_output-y
    sigmodp_fo=activation_derivative(fprop_output)
    output_smalldelta = output_error * sigmodp_fo
    ##Errores de hLayer
    hlayer_error = output_smalldelta.dot(thetas_dict['Theta Z_O'].T)
    sigmoidp_hl = activation_derivative(fprop_Z2)
    hlayer_smalldelta = hlayer_error* sigmoidp_hl*ld
    if debug:
        pass
        #np.savetxt('deltas.txt', output_smalldelta)
        #print("Smalldelta:",hlayer_smalldelta, "\nError", hlayer_error,"\nSprime", sigmoidp_hl)
        #print("Output Smalldelta:",output_smalldelta, "\noError", output_error,"\nSprime", sigmodp_fo)
    
    ##Ajustes a theta
    #Creemos un nuevo dict para no modificar el original (esto se ignorara si provoca problemas de memoria)
    if debug:
        print("ThetasIZ:",thetas_dict['Theta I_Z'][1])
        print("ThetasZO:",thetas_dict['Theta Z_O'][1])
        
    #print("Before:",thetas_dict['Theta I_Z'][1])
    #print((np.amax(thetas_dict['Theta I_Z'])))
    thetas_dict['Theta I_Z'] += (X.T.dot(hlayer_smalldelta))
    thetas_dict['Theta Z_O'] += fprop_Z2.T.dot(output_smalldelta)
    thetas_dict['Theta Z_O'] = thetas_dict['Theta Z_O']/(np.amax(thetas_dict['Theta Z_O']))
    #print("After:",thetas_dict['Theta I_Z'][1])

    

## Definicion de capa de entrada
En este caso utilizamos la suma de los valores de cada linea en la imagen para reducir la cantidad de neuronas en cada capa.

In [1]:
##Definamos la neurona de entrada.
#Mi idea es utilizar la suma de los valores de cada pixel en cada linea como mi input layer, reduciendo de 784 neuronas a 28.
##Entonces cargamos nuestros datasets de entrenamiento
from utils import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')

#X_train = X_train[:15000]
#print(X_train)
#X_train = X_train/(np.amax(X_train))
#print(X_train)

#Resheapeamos a y_train para que sea (60k, 1), si no da error.

y_train = np.reshape(y_train, (y_train.shape[0], 1))
#y_train = y_train[:15000]
y_train_new = np.zeros((y_train.shape[0], 10))
for i in range(len(y_train_new)):
    val = y_train[i][0]
    y_train_new[i][val] = 1
    
#print(y_train_new)
##Convertimos nuestro X_train de 784 columnas a un X_train_reduced de 28:
X_train_reduced = np.zeros((X_train.shape[0], 28))
for line in X_train:
    X_train_reduced[line] = (np.sum(line.reshape(-1, 28),axis=1))
    


NameError: name 'np' is not defined

In [5]:
#Probemos que hace nuestra funcion feed forward con lo que tenemos.
#fprop_res = forward_propagate(X_train_reduced,  sigmoid)
#backprop(X_train_reduced, y_train,  fprop_res, sigmoid_derivative, debug=True)
#fprop_res = forward_propagate(X_train_reduced,  sigmoid)

#print(thetas_dict['Theta Z_O'][1])
#Funcionan, creo, entonces probemos repetirlo n veces

ThetasIZ: [0.68949079 0.96295812 0.03855362 0.84317888 0.53962812 0.56846409
 0.79099495]
ThetasZO: [0.9230709  0.69257172 0.31802644 0.56884925 0.3973465  0.25634895
 0.08015698 0.71543253 0.41780387 0.57965382]


## Funcion para entrenar la red

In [6]:
def train(X, y, iter=1000, verbose_div=1000,  activation=sigmoid, activation_derivative = sigmoid_derivative):
    debug = True
    for i in range(iter):
        fprop_res = forward_propagate(X,  activation)
        backprop(X, y,  fprop_res, sigmoid_derivative, debug)
        debug = False
        if (i % verbose_div == 0) or (i == iter) :
            print("Expected output of first 10 samples...\n")
            print(y[:10])
            print("Obtained output of first 10 samples...\n")
            print(fprop_res['Activation'])
            print("Cost using mean sum squared:\n")
            print(y - fprop_res['Activation'])
    

In [7]:
train(X_train_reduced, y_train_new, iter=1000,verbose_div=100)

ThetasIZ: [0.68949079 0.96295812 0.03855362 0.84317888 0.53962812 0.56846409
 0.79099495]
ThetasZO: [1.73888101 1.63489185 1.36475662 1.93234828 2.14861396 2.1495634
 1.00010609 2.12797675 2.22809699 1.29551472]
Expected output of first 10 samples...

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
Obtained output of first 10 samples...

[[0.99999483 0.99998929 0.99992902 ... 0.99999966 0.99999983 0.99988479]
 [0.99999483 0.99998929 0.99992902 ... 0.99999966 0.99999983 0.99988479]
 [0.99999483 0.99998929 0.99992902 ... 0.99999966 0.99999983 0.99988479]
 ...
 [0.99773128 0.99673818 0.99164536 ... 0.99941776 0.99958976 0.98937959]
 [0.99773128 0.99673818 0.99164536 ... 0.99941776 0.99958976 0.98937959]
 [0.997

Expected output of first 10 samples...

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
Obtained output of first 10 samples...

[[0.99908824 0.99908838 0.99908874 ... 0.9990886  0.99908867 0.99908895]
 [0.99908824 0.99908838 0.99908874 ... 0.9990886  0.99908867 0.99908895]
 [0.99908824 0.99908838 0.99908874 ... 0.9990886  0.99908867 0.99908895]
 ...
 [0.97067672 0.97067893 0.97068445 ... 0.97068224 0.97068335 0.97068777]
 [0.97067672 0.97067893 0.97068445 ... 0.97068224 0.97068335 0.97068777]
 [0.97067672 0.97067893 0.97068445 ... 0.97068224 0.97068335 0.97068777]]
Cost using mean sum squared:

[[-9.99088242e-01 -9.99088383e-01 -9.99088737e-01 ... -9.99088595e-01
  -9.99088666e-01  9.11051194e-04]
 [ 9.11758

## Creamos los sets de prueba con la misma forma que el de entrenamiento

In [8]:
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')
y_test = np.reshape(y_test, (y_test.shape[0], 1))
y_test_new = np.zeros((y_test.shape[0], 10))
for i in range(len(y_test_new)):
    val = y_test[i][0]
    y_test_new[i][val] = 1
    

X_test_reduced = np.zeros((X_test.shape[0], 28))
for line in X_train:
    X_test_reduced[line] = (np.sum(line.reshape(-1, 28),axis=1))
print(X_test.shape, y_test_new.shape)
np.savetxt("xred.txt", X_test_reduced)


(10000, 784) (10000, 10)


## Se prueban los valores y se normaliza el output para poner encontrar cuales cazan con el valor esperado.

In [9]:
test_values = forward_propagate(X_test_reduced)
activated = test_values['Activation']
activated_norm = np.zeros(activated.shape)
for i in range(len(activated)):
    activated_norm[i] = activated[i]/np.amax(activated[i])

